#### Feature extraction experiments
This experiments are intended to test the performance of the model when using the feature extraction technique. The idea is to use the pre-trained model as a feature extractor and then train a new model using the extracted features. 

The features area extracted ahead of time and saved for performance reasons. The caveat is that we can not use the data augmentation techniques when extracting the features.

#### Vit Train on Features extracted
* Create a new Dataset with the extracted features
* Create a new model
* Train the model

In [1]:
from drecg.data.utils import FeaturesDataset
train_dataset = FeaturesDataset('features_ext_vit/train_features.pt')
train_dataset_augmented = FeaturesDataset('features_ext_vit/train_features_augmented.pt')
validation_dataset = FeaturesDataset('features_ext_vit/validation_features.pt')
test_dataset = FeaturesDataset('features_ext_vit/test_features.pt')


In [3]:
from torch.utils.data import DataLoader
train_feat_dataloader = DataLoader(train_dataset_augmented, batch_size=32, shuffle=True, num_workers=4)
test_feat_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=False, num_workers=4)
validation_dataloader = DataLoader(validation_dataset, batch_size=32, shuffle=False, num_workers=4)

In [4]:
from drecg.models.feat_extraction import DiffFeatureDetectorParam, LFullModel
model = DiffFeatureDetectorParam(num_hidden=1,
                                 hidden_units=128,
                                 features_dropout=0.4,
                                 hidden_dropout=0.3)
model = LFullModel(model, optimizer="AdamW", lr=0.0001)

In [5]:
import mlflow
exp_name = 'Feature Extraction VIT + Dense Difference'
mlflow.set_tracking_uri("http://127.0.0.1:5000")
# experiment_id = mlflow.create_experiment(
#     exp_name,
#     tags={"version": "v1", "type": "vit_features_concat_dense"},
# )

experiment = mlflow.set_experiment(exp_name)

In [ ]:
from drecg.training.loops import train_lightning
import torch
from mlflow import log_param, log_artifacts
log_param("num_layers", 1)
log_param("num_neurons", 128)
log_param("combine_method", "difference")
log_param("optimizer", "AdamW")
log_param("lr", 0.0001)
log_param("batch_size", 32)
log_param("num_epochs", 55)
log_param("augmentation_amount", "200%")
trainer = train_lightning(model, train_feat_dataloader, validation_dataloader, test_feat_dataloader, epochs=55)

torch.save(model, 'saved_models/candidate2/model.pt')
log_artifacts('saved_models/candidate2')

In [7]:

torch.save(model, 'saved_models/candidate2/model.pt')
log_artifacts('saved_models/candidate2')

In [8]:
m = torch.load('saved_models/candidate2/model.pt')

In [9]:
m

LFullModel(
  (model): DiffFeatureDetectorParam(
    (cls_layer): Sequential(
      (features_dropout): Dropout(p=0.4, inplace=False)
      (linear_0): Linear(in_features=1024, out_features=128, bias=True)
      (tanh_0): Tanh()
      (hidden_dropout_0): Dropout(p=0.3, inplace=False)
      (linear_out): Linear(in_features=128, out_features=1, bias=True)
    )
  )
  (loss_fn): BCEWithLogitsLoss()
  (train_acc): Accuracy()
  (valid_acc): Accuracy()
  (test_acc): Accuracy()
)

In [10]:
m2 = torch.load('candidate_model0.pt')

In [11]:
m2

LFullModel(
  (model): DiffFeatureDetector(
    (cls_layer): Sequential(
      (0): Dropout(p=0.35, inplace=False)
      (1): Linear(in_features=1024, out_features=128, bias=True)
      (2): ReLU()
      (3): Dropout(p=0.5, inplace=False)
      (4): Linear(in_features=128, out_features=1, bias=True)
    )
  )
  (loss_fn): BCEWithLogitsLoss()
  (train_acc): Accuracy()
  (valid_acc): Accuracy()
  (test_acc): Accuracy()
)

In [12]:

import pytorch_lightning as pl
trainer = pl.Trainer(accelerator='gpu', gpus=1)

/home/daniel/mambaforge/envs/recg/lib/python3.9/site-packages/pytorch_lightning/trainer/connectors/accelerator_connector.py:445: LightningDeprecationWarning: Setting `Trainer(gpus=1)` is deprecated in v1.7 and will be removed in v2.0. Please use `Trainer(accelerator='gpu', devices=1)` instead.
  rank_zero_deprecation(
/home/daniel/mambaforge/envs/recg/lib/python3.9/site-packages/pytorch_lightning/loops/utilities.py:93: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [15]:
trainer.test(model=m2, dataloaders=test_feat_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      accuracy:test         0.8930636048316956
        loss:test            0.332894891500473
          step                      1.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'loss': {'test': tensor(0.3329, device='cuda:0')},
  'accuracy': {'test': tensor(0.8931, device='cuda:0')},
  'step': 1.0}]

In [16]:
m3 = torch.load('saved_models/current/model.pt')

In [19]:
trainer.test(model=m3, dataloaders=test_feat_dataloader)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      accuracy:test         0.8901734352111816
        loss:test           0.35187745094299316
          step                      1.0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'loss': {'test': tensor(0.3519, device='cuda:0')},
  'accuracy': {'test': tensor(0.8902, device='cuda:0')},
  'step': 1.0}]